# MNIST KNN Graph

In [1]:
import argparse
import os.path as osp
import random
import time

import graphlearning as gl
import numpy as np
import pygsp
import rustworkx as rwx
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
import pandas as pd 
import os

from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid
from torch_geometric.logging import log
from torch_geometric.nn import GCNConv
import sys
sys.path.append(os.path.abspath(os.path.join('..')))
from src import get_num_edges_from_adj_matrix, generate_train_and_test_mask, GCN, eval_model, spectral_sparsify, mb_sparsify, thresh_sparsify


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

## Loading the dataset

In [3]:
ds_name = "mnist"
print(f"Benchmarking: {ds_name} dataset")

num_features = 784
num_classes = 10
print(f"    > {num_features} features")
print(f"    > {num_classes} classes")

# Loading the dataset :
source_data, labels = gl.datasets.load("mnist")

N = source_data.shape[0]

Benchmarking: mnist dataset
    > 784 features
    > 10 classes


## Subsampling the dataset

In [4]:
subsample_size = 2000
N = subsample_size
print(f"    > Running on a subsample of {subsample_size} Nodes")
indices = np.arange(0, source_data.shape[0], 1, dtype=int)
np.random.shuffle(indices)
chosen_indices = indices[:subsample_size]
source_data = source_data[chosen_indices, :]
labels = labels[chosen_indices]

K_range = [i for i in range(10,101,10)]
N_EPOCHS = 100

    > Running on a subsample of 2000 Nodes


## Full graph

In [26]:
#stats for the full graph
full_graph_training_time = np.zeros(len(K_range))
full_graph_training_time_per_epoch = np.zeros(len(K_range))
full_graph_accuracies = np.zeros(((len(K_range), N_EPOCHS)))
full_graph_best_accuracies = np.zeros(len(K_range))
full_graph_num_edges = np.zeros(len(K_range))
full_graph_sparsification_time = np.zeros(len(K_range))

In [27]:
for k_idx, k in enumerate(K_range):
    print(f"contructing the knn weightmatrix for k = {k}...")
    start_knn_weight_matrix_time = time.time()
    knn_weight_matrix = gl.weightmatrix.knn(source_data, k=k)
    knn_weight_matrix_time = time.time() - start_knn_weight_matrix_time
    full_graph_sparsification_time[k_idx] = knn_weight_matrix_time 
    print(f"done in {knn_weight_matrix_time:.4f}")
    print(f"the graph has {get_num_edges_from_adj_matrix(knn_weight_matrix.toarray())} edges")       
    full_graph_num_edges[k_idx] = get_num_edges_from_adj_matrix(knn_weight_matrix.toarray()) 
    train_mask, test_mask = generate_train_and_test_mask(N)
    
    full_graph_x = torch.tensor(source_data, dtype=torch.float)
    full_graph_y = torch.tensor(labels, dtype=torch.long)
    full_graph_edge_index = (
        torch.tensor(knn_weight_matrix.toarray()).nonzero().t().contiguous()
    )
    full_graph_data = Data(x=full_graph_x, edge_index=full_graph_edge_index, y=full_graph_y).to(device)
    # NOTE: des num_nodes update after having cut the dataset to get the subsample ?
    full_graph_model = GCN(num_features, 16, num_classes).to(device)
    
    full_graph_optimizer = torch.optim.Adam([
        dict(params=full_graph_model.conv1.parameters(), weight_decay=5e-4),
        dict(params=full_graph_model.conv2.parameters(), weight_decay=0)
    ], lr=0.01)  # Only perform weight-decay on first convolution.
    
    best_val_acc, accs, median_time_per_epoch, overall_time = eval_model(full_graph_model, 
                                                                            full_graph_data,
                                                                            train_mask,
                                                                            test_mask, 
                                                                            full_graph_optimizer,
                                                                            N_EPOCHS)

contructing the knn weightmatrix for k = 10...
done in 0.8230
the graph has 14581 edges
Epoch: 001, Loss: 2.3442, Train: 0.1910, Test: 0.1820
Epoch: 010, Loss: 0.8910, Train: 0.8140, Test: 0.8200
Epoch: 020, Loss: 0.5381, Train: 0.8730, Test: 0.8700
Epoch: 030, Loss: 0.4452, Train: 0.8940, Test: 0.8890
Epoch: 040, Loss: 0.3933, Train: 0.9120, Test: 0.9020
Epoch: 050, Loss: 0.3819, Train: 0.9150, Test: 0.9030
Epoch: 060, Loss: 0.3372, Train: 0.9180, Test: 0.9010
Epoch: 070, Loss: 0.3061, Train: 0.9250, Test: 0.9030
Epoch: 080, Loss: 0.2942, Train: 0.9310, Test: 0.9060
Epoch: 090, Loss: 0.2653, Train: 0.9380, Test: 0.9140
Epoch: 100, Loss: 0.2951, Train: 0.9350, Test: 0.9090
contructing the knn weightmatrix for k = 20...
done in 0.8006
the graph has 28776 edges
Epoch: 001, Loss: 2.3656, Train: 0.2220, Test: 0.2010
Epoch: 010, Loss: 0.9416, Train: 0.7930, Test: 0.8060
Epoch: 020, Loss: 0.6100, Train: 0.8390, Test: 0.8470
Epoch: 030, Loss: 0.5051, Train: 0.8750, Test: 0.8720
Epoch: 040, Lo

KeyboardInterrupt: 

# Spectral Sparsified Graph

In [9]:
#stats for the spectral sparsification
spectral_training_time = np.zeros(len(K_range))
spectral_graph_training_time_per_epoch = np.zeros(len(K_range))
spectral_accuracies = np.zeros(((len(K_range), N_EPOCHS))) 
spectral_best_accuracies = np.zeros(len(K_range))
spectral_num_edges = np.zeros(len(K_range))
spectral_sparsification_time = np.zeros(len(K_range))

In [12]:
for k_idx, k in enumerate(K_range):
    print(f"constructing the spectral sparisfied graph using effective resistance")
    knn_weight_matrix = gl.weightmatrix.knn(source_data, k=k)
    start_spectral_sparsify = time.time() 
    spectral_sparse_knn_weight_matrix = spectral_sparsify(np.nan_to_num((1/knn_weight_matrix.todense())-1, posinf = 0, neginf = 0, nan = 0))
    spectral_sparsify_time = time.time() - start_spectral_sparsify
    print(f"done in {spectral_sparsify_time:.4f}s")
    spectral_sparsification_time[k_idx] = spectral_sparsify_time
    spectral_num_edges[k_idx] = get_num_edges_from_adj_matrix(spectral_sparse_knn_weight_matrix.toarray())
    train_mask, test_mask = generate_train_and_test_mask(N)

        
    print(f'-------------------------- Running the model Spectral sparsified |E| = {spectral_num_edges[k_idx]}, with K = {k} --------------------------')
    # Running the model on the spectral sparsified graph
    spectral_x = torch.tensor(source_data, dtype=torch.float)
    spectral_y = torch.tensor(labels, dtype=torch.long)
    spectral_edge_index = (
        torch.tensor(spectral_sparse_knn_weight_matrix.toarray()).nonzero().t().contiguous()
    )
    spectral_data = Data(x=spectral_x, edge_index=spectral_edge_index, y=spectral_y).to(device)
    # NOTE: des num_nodes update after having cut the dataset to get the subsample ?
    spectral_model = GCN(num_features, 16, num_classes).to(device)

    spectral_optimizer = torch.optim.Adam([
        dict(params=spectral_model.conv1.parameters(), weight_decay=5e-4),
        dict(params=spectral_model.conv2.parameters(), weight_decay=0)
    ], lr=0.01)  # Only perform weight-decay on first convolution.

    best_val_acc, accs, median_time_per_epoch, overall_time = eval_model(spectral_model, 
                                                                            spectral_data,
                                                                            train_mask,
                                                                            test_mask, 
                                                                            spectral_optimizer,
                                                                            N_EPOCHS)

constructing the spectral sparisfied graph using effective resistance


/var/folders/4l/t1p1zlw56kx34l_phf3ncrg40000gn/T/ipykernel_21611/3197016560.py:5: RuntimeWarning: divide by zero encountered in divide
  spectral_sparse_knn_weight_matrix = spectral_sparsify(np.nan_to_num((1/knn_weight_matrix.todense())-1, posinf = 0, neginf = 0, nan = 0))


done in 2.2693s
-------------------------- Running the model Spectral sparsified |E| = 10888.0, with K = 10 --------------------------
Epoch: 001, Loss: 2.3533, Train: 0.4700, Test: 0.4520
Epoch: 010, Loss: 1.0550, Train: 0.7660, Test: 0.7550
Epoch: 020, Loss: 0.6605, Train: 0.8590, Test: 0.8350
Epoch: 030, Loss: 0.5671, Train: 0.8830, Test: 0.8530
Epoch: 040, Loss: 0.4842, Train: 0.8930, Test: 0.8600
Epoch: 050, Loss: 0.4316, Train: 0.9090, Test: 0.8700
Epoch: 060, Loss: 0.3904, Train: 0.9100, Test: 0.8830
Epoch: 070, Loss: 0.4000, Train: 0.9130, Test: 0.8760
Epoch: 080, Loss: 0.3812, Train: 0.9180, Test: 0.8860
Epoch: 090, Loss: 0.3292, Train: 0.9180, Test: 0.8850
Epoch: 100, Loss: 0.3344, Train: 0.9190, Test: 0.8820
constructing the spectral sparisfied graph using effective resistance
done in 2.7967s
-------------------------- Running the model Spectral sparsified |E| = 15446.0, with K = 20 --------------------------
Epoch: 001, Loss: 2.2930, Train: 0.2920, Test: 0.2800
Epoch: 010, 

/Users/yofi/miniconda3/lib/python3.11/site-packages/pygsp/reduction.py:118: RuntimeWarning: divide by zero encountered in divide
  per_spin_weights = weights / (q * Pe)
/Users/yofi/miniconda3/lib/python3.11/site-packages/pygsp/reduction.py:122: RuntimeWarning: invalid value encountered in multiply
  new_weights = counts * per_spin_weights
2024-06-04 15:13:11,293:[WARNING](pygsp.graphs.graph.check_weights): There is a NaN value in the weight matrix!
2024-06-04 15:13:12,121:[WARNING](pygsp.graphs.graph.check_weights): There is a NaN value in the weight matrix!
2024-06-04 15:13:12,363:[WARNING](pygsp.graphs.graph.check_weights): There is a NaN value in the weight matrix!


done in 4.9582s
-------------------------- Running the model Spectral sparsified |E| = 27080.0, with K = 90 --------------------------
Epoch: 001, Loss: 2.2862, Train: 0.1760, Test: 0.1640
Epoch: 010, Loss: 1.5151, Train: 0.5790, Test: 0.5850
Epoch: 020, Loss: 1.1616, Train: 0.6400, Test: 0.6480
Epoch: 030, Loss: 1.0027, Train: 0.7030, Test: 0.7000
Epoch: 040, Loss: 0.9559, Train: 0.7100, Test: 0.7180
Epoch: 050, Loss: 0.9094, Train: 0.7190, Test: 0.7160
Epoch: 060, Loss: 0.8864, Train: 0.7270, Test: 0.7340
Epoch: 070, Loss: 0.8291, Train: 0.7470, Test: 0.7460
Epoch: 080, Loss: 0.8173, Train: 0.7580, Test: 0.7500
Epoch: 090, Loss: 0.8132, Train: 0.7630, Test: 0.7580
Epoch: 100, Loss: 0.7672, Train: 0.7770, Test: 0.7520
constructing the spectral sparisfied graph using effective resistance
done in 4.8560s
-------------------------- Running the model Spectral sparsified |E| = 22437.0, with K = 100 --------------------------
Epoch: 001, Loss: 2.3242, Train: 0.1440, Test: 0.1270
Epoch: 010,

# Metric Backbone

In [5]:
#stats for the metric backbone
mb_training_time = np.zeros(len(K_range))
mb_graph_training_time_per_epoch = np.zeros(len(K_range))
mb_accuracies = np.zeros(((len(K_range), N_EPOCHS)))
mb_best_accuracies = np.zeros(len(K_range))
mb_num_edges = np.zeros(len(K_range))
mb_sparsification_time = np.zeros(len(K_range))

In [6]:
for k_idx, k in enumerate(K_range):
    knn_weight_matrix = gl.weightmatrix.knn(source_data, k=k)
    print(f"constructing the metric backbone graph")
    start_mb_sparsify = time.time()
    mb_sparse_knn_weight_matrix = mb_sparsify(np.nan_to_num((1/knn_weight_matrix.todense())-1, posinf = 0, neginf = 0))
    mb_sparsify_time = time.time() - start_mb_sparsify
    print(f"done in {mb_sparsify_time:.4f}s")
    mb_sparsification_time[k_idx] = mb_sparsify_time
    mb_num_edges[k_idx] = get_num_edges_from_adj_matrix(mb_sparse_knn_weight_matrix)
    train_mask, test_mask = generate_train_and_test_mask(N) 
    
    print(f'-------------------------- Running the model Metric Backbone sparsified |E| = {mb_num_edges[k_idx]}, with K = {k} --------------------------')
    # Running the model on the metric backbone sparsified graph
    mb_x = torch.tensor(source_data, dtype=torch.float)
    mb_y = torch.tensor(labels, dtype=torch.long)
    mb_edge_index = (
        torch.tensor(mb_sparse_knn_weight_matrix).nonzero().t().contiguous()
    )
    mb_data = Data(x=mb_x, edge_index=mb_edge_index, y=mb_y).to(device)
    # NOTE: des num_nodes update after having cut the dataset to get the subsample ?
    mb_model = GCN(num_features, 16, num_classes).to(device)
    
    mb_optimizer = torch.optim.Adam([
        dict(params=mb_model.conv1.parameters(), weight_decay=5e-4),
        dict(params=mb_model.conv2.parameters(), weight_decay=0)
    ], lr=0.01)  # Only perform weight-decay on first convolution.
    
    best_val_acc, accs, median_time_per_epoch, overall_time = eval_model(mb_model, 
                                                                            mb_data,
                                                                            train_mask,
                                                                            test_mask, 
                                                                            mb_optimizer,
                                                                            N_EPOCHS)
    
    print(f"median time per epoch: {median_time_per_epoch:.4f}s")
    mb_best_accuracies[k_idx] = best_val_acc
    mb_accuracies[k_idx] = accs
    mb_graph_training_time_per_epoch[k_idx] = median_time_per_epoch
    mb_training_time[k_idx] = overall_time

constructing the metric backbone graph


/var/folders/4l/t1p1zlw56kx34l_phf3ncrg40000gn/T/ipykernel_22586/306633343.py:5: RuntimeWarning: divide by zero encountered in divide
  mb_sparse_knn_weight_matrix = mb_sparsify(np.nan_to_num((1/knn_weight_matrix.todense())-1, posinf = 0, neginf = 0))


done in 0.2729s
-------------------------- Running the model Metric Backbone sparsified |E| = 7514.0, with K = 10 --------------------------
Epoch: 001, Loss: 2.3656, Train: 0.2930, Test: 0.3120
Epoch: 010, Loss: 1.0664, Train: 0.7350, Test: 0.7220
Epoch: 020, Loss: 0.7241, Train: 0.8180, Test: 0.8170
Epoch: 030, Loss: 0.6069, Train: 0.8540, Test: 0.8460
Epoch: 040, Loss: 0.5215, Train: 0.8770, Test: 0.8650
Epoch: 050, Loss: 0.4704, Train: 0.8890, Test: 0.8780
Epoch: 060, Loss: 0.4510, Train: 0.8920, Test: 0.8820
Epoch: 070, Loss: 0.4428, Train: 0.9020, Test: 0.8820
Epoch: 080, Loss: 0.4155, Train: 0.9050, Test: 0.8800
Epoch: 090, Loss: 0.4146, Train: 0.9080, Test: 0.8920
Epoch: 100, Loss: 0.4145, Train: 0.9110, Test: 0.8880
median time per epoch: 0.0146s
constructing the metric backbone graph
done in 0.4091s
-------------------------- Running the model Metric Backbone sparsified |E| = 9903.0, with K = 20 --------------------------
Epoch: 001, Loss: 2.4117, Train: 0.2880, Test: 0.3010


In [46]:
knn_weight_matrix.nonzero()


(array([   0,    0,    0, ..., 1999, 1999, 1999], dtype=int32),
 array([  13,   40,   61, ..., 1982, 1983, 1988], dtype=int32))

# Threshold Sparsified

In [7]:
#stats for the threshold
thresh_training_time = np.zeros(len(K_range))
thresh_graph_training_time_per_epoch = np.zeros(len(K_range))
thresh_accuracies = np.zeros(((len(K_range), N_EPOCHS)))
thresh_best_accuracies = np.zeros(len(K_range))
thresh_num_edges = np.zeros(len(K_range))
thresh_sparsification_time = np.zeros(len(K_range))

In [8]:
for k_idx, k in enumerate(K_range):
    knn_weight_matrix = gl.weightmatrix.knn(source_data, k=k)
    print(f"constructing the threshold graph")
    start_thresh_sparsify = time.time()
    dist_matrix = mb_sparsify(np.nan_to_num((1/knn_weight_matrix.todense())-1, posinf = 0, neginf = 0, nan = 0))
    thresh_sparse_knn_weight_matrix = thresh_sparsify(dist_matrix, mb_num_edges[k_idx])
    thresh_sparsify_time = time.time() - start_thresh_sparsify
    print(f"done in {thresh_sparsify_time:.4f}s")
    thresh_sparsification_time[k_idx] = thresh_sparsify_time
    thresh_num_edges[k_idx] = get_num_edges_from_adj_matrix(thresh_sparse_knn_weight_matrix)
    
    print(f'-------------------------- Running the model Threshold sparsified |E| = {thresh_num_edges[k_idx]}, with K = {k} --------------------------')
    # Running the model on the threshold sparsified graph
    thresh_x = torch.tensor(source_data, dtype=torch.float)
    thresh_y = torch.tensor(labels, dtype=torch.long)
    thresh_edge_index = (
        torch.tensor(thresh_sparse_knn_weight_matrix).nonzero().t().contiguous()
    )
    thresh_data = Data(x=thresh_x, edge_index=thresh_edge_index, y=thresh_y).to(device)
    # NOTE: des num_nodes update after having cut the dataset to get the subsample ?
    thresh_model = GCN(num_features, 16, num_classes).to(device)
    
    thresh_optimizer = torch.optim.Adam([
        dict(params=thresh_model.conv1.parameters(), weight_decay=5e-4),
        dict(params=thresh_model.conv2.parameters(), weight_decay=0)
    ], lr=0.01)  # Only perform weight-decay on first convolution.
    
    best_val_acc, accs, median_time_per_epoch, overall_time = eval_model(thresh_model, 
                                                                            thresh_data,
                                                                            train_mask,
                                                                            test_mask, 
                                                                            thresh_optimizer,
                                                                            N_EPOCHS)
    

constructing the threshold graph


/var/folders/4l/t1p1zlw56kx34l_phf3ncrg40000gn/T/ipykernel_22586/499155617.py:5: RuntimeWarning: divide by zero encountered in divide
  dist_matrix = mb_sparsify(np.nan_to_num((1/knn_weight_matrix.todense())-1, posinf = 0, neginf = 0, nan = 0))


done in 0.3430s
-------------------------- Running the model Threshold sparsified |E| = 7514.0, with K = 10 --------------------------
Epoch: 001, Loss: 2.2838, Train: 0.2410, Test: 0.2310
Epoch: 010, Loss: 1.1800, Train: 0.7170, Test: 0.6720
Epoch: 020, Loss: 0.7521, Train: 0.8460, Test: 0.8300
Epoch: 030, Loss: 0.5916, Train: 0.8720, Test: 0.8540
Epoch: 040, Loss: 0.5235, Train: 0.8950, Test: 0.8570
Epoch: 050, Loss: 0.4862, Train: 0.9010, Test: 0.8570
Epoch: 060, Loss: 0.4539, Train: 0.9070, Test: 0.8650
Epoch: 070, Loss: 0.4364, Train: 0.9180, Test: 0.8710
Epoch: 080, Loss: 0.3854, Train: 0.9200, Test: 0.8770
Epoch: 090, Loss: 0.3660, Train: 0.9270, Test: 0.8740
Epoch: 100, Loss: 0.3970, Train: 0.9200, Test: 0.8710
constructing the threshold graph
done in 0.4493s
-------------------------- Running the model Threshold sparsified |E| = 9903.0, with K = 20 --------------------------
Epoch: 001, Loss: 2.4213, Train: 0.3050, Test: 0.3130
Epoch: 010, Loss: 1.3698, Train: 0.6330, Test: 0.